In [154]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder, RobustScaler

import importlib
try:
    importlib.reload(utils) # reload module
except NameError:
    pass

# Read Dataset

In [110]:
path= 'input/dataset.csv'
df= pd.read_csv(path, delimiter= '\t')

df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


# Clean Data

In [111]:
df_clean= utils.clean_df(df)

# Pre-EDA

# Preprocessing

In [155]:
all_cols= df_clean.columns
num_cols= df_clean.select_dtypes(include=['int64', 'float64']).columns
cat_cols= [c for c in all_cols if c not in num_cols]

# encode cat features
encoder= OneHotEncoder(sparse_output= False)
df_cat= pd.DataFrame(
    data= encoder.fit_transform(df_clean[cat_cols]), 
    columns= encoder.get_feature_names_out()
)

# take remaining numerical columns
df_num= df_clean[num_cols]

# concat all together to one DF again
df_onehot= pd.concat([df_num, df_cat], axis= 1)

In [156]:
# scale all features
scaler= RobustScaler()

df_scaled= pd.DataFrame(
    data= scaler.fit_transform(df_onehot), 
    columns= df_onehot.columns
)

# Segmentation

# Post-EDA

# Summary and Recommendation